In [1]:
import random
import numpy as np
import pandas as pd

In [2]:
team_rankings = {}

#based on 2021 ladder position
team_rankings["Adelaide"]=15
team_rankings["Brisbane"]=4
team_rankings["Carlton"]=13
team_rankings["Collingwood"]=17
team_rankings["Essendon"]=8
team_rankings["Fremantle"]=11
team_rankings["Geelong"]=3
team_rankings["Gold Coast"]=16
team_rankings["GWS"]=7
team_rankings["Hawthorn"]=14
team_rankings["Melbourne"]=1
team_rankings["North Melbourne"]=18
team_rankings["Port Adelaide"]=2
team_rankings["Richmond"]=12
team_rankings["St Kilda"]=10
team_rankings["Sydney"]=6
team_rankings["West Coast"]=9
team_rankings["Western Bulldogs"]=5

In [3]:
team_grounds = {}

team_grounds["Adelaide"]="Adelaide Oval"
team_grounds["Brisbane"]="Gabba"
team_grounds["Carlton"]="MCG"
team_grounds["Collingwood"]="MCG"
team_grounds["Essendon"]="MCG"
team_grounds["Fremantle"]="Optus Stadium"
team_grounds["Geelong"]="GMHBA Stadium"
team_grounds["Gold Coast"]="Metricon Stadium"
team_grounds["GWS"]="GIANTS Stadium"
team_grounds["Hawthorn"]="MCG"
team_grounds["Melbourne"]="MCG"
team_grounds["North Melbourne"]="Marvel Stadium"
team_grounds["Port Adelaide"]="Adelaide Oval"
team_grounds["Richmond"]="MCG"
team_grounds["St Kilda"]="Marvel Stadium"
team_grounds["Sydney"]="SCG"
team_grounds["West Coast"]="Optus Stadium"
team_grounds["Western Bulldogs"]="Marvel Stadium"

In [4]:
timeslots = {}

timeslots[1] = 'Thursday 7.50pm'
timeslots[2] = 'Friday 7.50pm'
timeslots[3] = 'Saturday 7.25pm'
timeslots[4] = 'Saturday 7.40pm'
timeslots[5] = 'Saturday 2.10pm'
timeslots[6] = 'Saturday 1.45pm'
timeslots[7] = 'Sunday 3.20pm'
timeslots[8] = 'Sunday 1.10pm'
timeslots[9] = 'Sunday 4.40pm'

In [5]:
teams = ["Adelaide","Brisbane","Carlton","Collingwood","Essendon","Fremantle","Geelong","Gold Coast","GWS",
        "Hawthorn","Melbourne","North Melbourne","Port Adelaide","Richmond","St Kilda","Sydney",
         "West Coast","Western Bulldogs"]

random.shuffle(teams)

In [6]:
def basic_rr(teams):
    s = {}
    s_per_team = {}
    h_count_per_team = {}
    ha_per_team = {}
    
    #if odd n_teams
    if len(teams) % 2 == 1: 
        teams = teams + [None]
    
    n = len(teams)
    
    for team in teams:
        s_per_team[team] = []
        h_count_per_team[team] = 0
        ha_per_team[team] = []
        
    mid = n//2
    
    for i in range(n-1):
        round=[]
        for j in range(mid):
            t1 = teams[j]
            t2 = teams[n-1-j]
            
            round.append((t1, "vs.", t2))
            
            s_per_team[t1].append(t2)
            s_per_team[t2].append(t1)
            h_count_per_team[t1] += 1
            ha_per_team[t1].append("Home")
            ha_per_team[t2].append("Away")
                
        s["Round " + str(i+1)] = round
        
        teams.insert(1,teams.pop())
        
    s_per_team = dict(sorted(s_per_team.items(), key=lambda x: x[0].lower()))
    h_count_per_team = dict(sorted(h_count_per_team.items(), key=lambda x: x[0].lower()))
    ha_per_team = dict(sorted(ha_per_team.items(), key=lambda x: x[0].lower()))
    
    return s, s_per_team, h_count_per_team, ha_per_team

In [7]:
def round_robin_fair(teams):
    """ Create a schedule for the teams in the list and return it"""
    s = {}
    s_per_team = {}
    h_count_per_team = {}
    ha_per_team = {}
    
    if len(teams) % 2 == 1: teams = teams + [None]
    # manipulate map (array of indexes for list) instead of list itself
    # this takes advantage of even/odd indexes to determine home vs. away
    n = len(teams)
    
    for team in teams:
        s_per_team[team] = []
        h_count_per_team[team] = 0
        ha_per_team[team] = []
        
    map = list(range(n))
    mid = n // 2
    for i in range(n-1):
        l1 = map[:mid]
        l2 = map[mid:]
        l2.reverse()
        #print("Round",i)
        round = []
        
        for j in range(mid):
            t1 = teams[l1[j]]
            t2 = teams[l2[j]]
            if j == 0 and i % 2 == 1:
                # flip the first match only, every other round
                # (this is because the first match always involves the last player in the list)
                round.append((t2,"vs.",t1))
                s_per_team[t2].append(t1)
                s_per_team[t1].append(t2)
                h_count_per_team[t2] += 1
                ha_per_team[t2].append("Home")
                ha_per_team[t1].append("Away")
            else:
                round.append((t1, "vs.", t2))
                s_per_team[t2].append(t1)
                s_per_team[t1].append(t2)
                h_count_per_team[t1] += 1
                ha_per_team[t1].append("Home")
                ha_per_team[t2].append("Away")
                
        s["Round " + str(i+1)] = round
        
        # rotate list by n/2, leaving last element at the end
        map = map[mid:-1] + map[:mid] + map[-1:]
        
    s_per_team = dict(sorted(s_per_team.items(), key=lambda x: x[0].lower()))
    h_count_per_team = dict(sorted(h_count_per_team.items(), key=lambda x: x[0].lower()))
    ha_per_team = dict(sorted(ha_per_team.items(), key=lambda x: x[0].lower()))
    
    return s, s_per_team, h_count_per_team, ha_per_team

In [8]:
def optimize_tv_ratings(schedule):
    
    new_schedule = {}
    new_schedule_chron = {}
    
    for round in schedule:
        temp_rating_list = []
        for game in schedule[round]:
            hometeam = game[0]
            awayteam = game[2]
            
            htr = team_rankings[hometeam]
            atr = team_rankings[awayteam]
            
            temp_rating_list.append(htr+atr)
        
        
        argsort_trl = np.argsort(temp_rating_list)
        
        newround=[]
        
        for i in range(len(temp_rating_list)):
            nextgame = argsort_trl[i]
            
            hometeam = schedule[round][nextgame][0]
            awayteam = schedule[round][nextgame][2]
            
            ground = team_grounds[hometeam]
            
            timeslot = timeslots[i+1]
            
            newround.append((hometeam,awayteam,ground,timeslot,str(temp_rating_list[nextgame]),i+1))
            
        new_schedule[round]=newround
    
    for round in new_schedule:
        
        chronround = []
        
        chronround.append(new_schedule[round][0])
        chronround.append(new_schedule[round][1])
        chronround.append(new_schedule[round][5])
        chronround.append(new_schedule[round][4])
        chronround.append(new_schedule[round][2])
        chronround.append(new_schedule[round][3])
        chronround.append(new_schedule[round][7])
        chronround.append(new_schedule[round][6])
        chronround.append(new_schedule[round][8])
        
    
        new_schedule_chron[round]=chronround
        
        
    return new_schedule_chron

In [9]:
def timeslots_non_opt(schedule):
    new_schedule = {}
    new_schedule_chron = {}
    
    for round in schedule:
        temp_rating_list = []
        for game in schedule[round]:
            hometeam = game[0]
            awayteam = game[2]
            
            htr = team_rankings[hometeam]
            atr = team_rankings[awayteam]
            
            temp_rating_list.append(htr+atr)
        
        
        newround=[]
        
        for i in range(len(temp_rating_list)):
            
            hometeam = schedule[round][i][0]
            awayteam = schedule[round][i][2]
            
            ground = team_grounds[hometeam]
            
            timeslot = timeslots[i+1]
            
            newround.append((hometeam,awayteam,ground,timeslot,str(temp_rating_list[i]),i+1))
            
        new_schedule[round]=newround
        
    for round in new_schedule:
        
        chronround = []
        
        chronround.append(new_schedule[round][0])
        chronround.append(new_schedule[round][1])
        chronround.append(new_schedule[round][5])
        chronround.append(new_schedule[round][4])
        chronround.append(new_schedule[round][2])
        chronround.append(new_schedule[round][3])
        chronround.append(new_schedule[round][7])
        chronround.append(new_schedule[round][6])
        chronround.append(new_schedule[round][8])
        
    
        new_schedule_chron[round]=chronround
        
        
    return new_schedule_chron

In [ ]:
def create_schedule_df(schedule):
    
    df_list = []
    cols = ['Round','Home Team','Away Team','Venue','Time','Rank','TimeRank']
    
    for round in schedule:
        df = pd.DataFrame(schedule[round],columns=['Home Team','Away Team','Venue','Time','Rank','TimeRank'])
        df['Round'] = round
        df = df[cols]
        
        df_list.append(df)
        
        
    full_df = pd.concat(df_list)
    
    return full_df

In [45]:
def create_partial_schedule_df(schedule):
    
    df_list = []
    cols = ['Round','Home Team','Away Team','Venue','Time','Rank','TimeRank']
    
    for round in schedule:
        df = pd.DataFrame(schedule[round],columns=['Home Team','Away Team','Venue','Time','Rank','TimeRank'])
        df['Round'] = round
        df = df[cols]
        
        df_list.append(df)
        
        
    full_df = pd.concat(df_list[:4])
    
    return full_df

In [ ]:
def obj_HAV(ha_perteam):
    counter=0
    for team in ha_perteam:
        for i in range(1,len(ha_perteam[team])):
            if ha_perteam[team][i] == ha_perteam[team][i-1]:
                counter += 1
    return counter

In [70]:
def obj_TV(df):
    df['TimeRankInv'] = 9/df['TimeRank']
    
    #note max team rank for a game is 35, so to invert Rank per game, take reciprocal of 35
    df['TV_Rank'] = df['TimeRankInv']* (35/df['Rank'].astype(int))
    
    return df['TV_Rank'].sum()

In [46]:
basic_sched, basic_per_team, basic_hcpt, basic_hapt = basic_rr(teams)

In [47]:
fair_sched, fair_per_team, fair_hcpt, fair_hapt = round_robin_fair(teams)

In [48]:
basic_sched_ts = timeslots_non_opt(basic_sched)

In [49]:
ha_fair_ts = timeslots_non_opt(fair_sched)

In [50]:
tv_opt_ts = optimize_tv_ratings(fair_sched)

In [51]:
basic_sched_df = create_schedule_df(basic_sched_ts)

In [52]:
basic_sched_df_partial = create_partial_schedule_df(basic_sched_ts)

In [53]:
ha_fair_df = create_schedule_df(ha_fair_ts)

In [54]:
ha_fair_df_partial = create_partial_schedule_df(ha_fair_ts)

In [55]:
tv_opt_df = create_schedule_df(tv_opt_ts)

In [56]:
tv_opt_df_partial = create_partial_schedule_df(tv_opt_ts)

In [57]:
basic_sched_df.drop(columns=['Rank','TimeRank']).to_csv('basic.csv',index=False)

In [58]:
basic_sched_df_partial.drop(columns=['Rank','TimeRank']).to_csv('basic_partial.csv',index=False)

In [59]:
ha_fair_df.drop(columns=['Rank','TimeRank']).to_csv('ha_fair.csv',index=False)

In [60]:
ha_fair_df_partial.drop(columns=['Rank','TimeRank']).to_csv('ha_fair_partial.csv',index=False)

In [61]:
tv_opt_df.drop(columns=['Rank','TimeRank']).to_csv('tv_opt.csv',index=False)

In [62]:
tv_opt_df_partial.drop(columns=['Rank','TimeRank']).to_csv('tv_opt_partial.csv',index=False)

In [72]:
#example of OBJECTIVE FUNCTION

#BASIC ROUND ROBIN
print("HAV",obj_HAV(basic_hapt))

print("TV Optimisation",obj_TV(basic_sched_df))

HAV 256
TV Optimisation 883.8090688421221


In [73]:
#FAIR HA
print("HAV",obj_HAV(fair_hapt))

print("TV Optimisation",obj_TV(ha_fair_df))

HAV 16
TV Optimisation 1078.8120632940954


In [74]:
#TV OPT
print("HAV",obj_HAV(fair_hapt))

print("TV Optimisation",obj_TV(tv_opt_df))

HAV 16
TV Optimisation 1419.5300692854257
